# L2: Information Extraction

In this lab you will implement and evaluate a simple system for information extraction. The task of the system is to read sentences and extract entity pairs of the form *x*&ndash;*y* where *x*&nbsp;is a person, *y*&nbsp;is an organisation, and *x* is the &lsquo;leader&rsquo; of&nbsp;*y*. Consider the following example sentence:

<blockquote>
Mr. Obama also selected Lisa Jackson to head the Environmental Protection Agency.
</blockquote>

From this sentence the system should extract the pair
```
("Lisa Jackson", "Environmental Protection Agency")
```

The system will have to solve the following sub-tasks:
* entity extraction &ndash; identifying mentions of person entities in text
* relation extraction &ndash; identifying instances of the &lsquo;is-leader-of&rsquo; relation

The data set for the lab consists of 62,010&nbsp;sentences from the [Groningen Meaning Bank](http://gmb.let.rug.nl) (release 2.2.0), an open corpus of English. To analyse the sentences you will use [spaCy](https://spacy.io/).

## Getting started

The first cell imports the Python module required for this lab.

In [1]:
import tm2

The data is contained in the following file:

In [2]:
#data_file = "C:/Users/Carles/Desktop/MasterStatistics-MachineLearning/Master_subjects/Text_Mining/Labs/Lab2/gmb.txt"
#data_file = "/home/carsa564/Desktop/TextMining/gmb.txt"

data_file = "/home/TDDE16/labs/l2/data/gmb.txt"


The `tm2` module defines a function `read_data` that returns an iterator over the lines in a file. You should use this function to read the data for this lab. Use the optional argument `n` to restrict the iteration to the first few lines of the file. Here is an example:

In [3]:
#data = open(data_file, "r", encoding="utf8")
for sentence in tm2.read_data(data_file, n=3):
    print(sentence)
        

Masked assailants with grenades and automatic weapons attacked a wedding party in southeastern Turkey, killing 45 people and wounding at least six others.
Turkish officials said the attack occurred Monday in the village of Bilge about 600 kilometers from Ankara.
The wounded were taken to the hospital in the nearby city of Mardin.


The next cell imports spaCy and loads its English language model.

In [4]:
import spacy

nlp = spacy.load('en', disable=['textcat'])

## Entity extraction

To implement the entity extraction part of your system, you do not need to do much, as you can use the full natural language processing power built into spaCy. The following code extracts the entities from the first 5&nbsp;sentences of the data:

In [5]:
#for i, doc in enumerate(nlp.pipe(data)):
#    for ent in doc.ents:
#        print("{}\t{}\t{}\t{}".format(ent.text, ent.start, ent.end, ent.label_))

for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=5))):
    for ent in doc.ents:
        print("{}\t{}\t{}\t{}".format(ent.text, ent.start, ent.end, ent.label_))

Turkey	13	14	GPE
45	16	17	CARDINAL
at least six	20	23	CARDINAL
Turkish	0	1	NORP
Monday	6	7	DATE
Bilge	11	12	GPE
about 600 kilometers	12	15	QUANTITY
Ankara	16	17	GPE
Mardin	12	13	GPE


Read the [section about named entities](https://spacy.io/usage/linguistic-features#section-named-entities) from spaCy&rsquo;s documentation to get some background on this. (Please note that we are using version&nbsp;1 of the spaCy library, which means that there may be slight differences in the usage. At the time of writing, the current version&nbsp;2 is not yet stable and fast enough for this lab.)

## Problem 1: Extract relevant pairs

The first problem that you will have to solve is to identify pairs of entities that are in the &lsquo;is-leader-of&rsquo; relation, as in the example above. There are many ways to do this, but for this lab it suffices to implement the strategy outlined in the section on [Relation Extraction](http://www.nltk.org/book/ch07.html#relation-extraction) in the book by Bird, Klein, and Loper (2009):

* look for all triples of the form $(X, \alpha, Y)$ where $X$ and $Y$ are named entities of type *person* and $\alpha$ is the intervening text
* write a regular expression to match just those instances of $\alpha$ that express the &lsquo;is-leader-of&rsquo; relation

You can restrict your attention to adjacent pairs of entities &ndash; that is, cases where $X$ precedes $Y$ and $\alpha$ does not contain other named entities.

Write a function `extract` that takes an analysed sentence (represented as a spaCy [`Doc`](https://spacy.io/api/doc) object) and yields pairs $(X, Y)$ of strings representing entity mentions predicted to be in the &lsquo;is-leader-of&rsquo; relation.

In [6]:
import nltk, re
def extract(doc):
    """Extract relevant relation instances from the specified document.
    
    Args:
        doc: The sentence as analysed by spaCy.
    Yields:
        Pairs of strings representing the extracted relation instances.
    """
    # TODO: Replace the following line with your own code
    leader = re.compile(r'.*(lead|command|direct|govern|head|manage|preside|supervis|chief|patron).*')
    person = None
    org= None
    ReturningString = []
    for ent in doc.ents:
        if ent.label_=="PERSON":
            person = ent
        else:
            if ent.label_ == "ORG" and person != None:
                org = ent
                intervening_words = doc.text.split(" ")[person.end:org.start]
                intervening_sentence = " ".join(intervening_words)
                if re.search(leader, intervening_sentence):
                    ReturningString.append((str(person), str(org)))
                    person = None
                    org = None
    return(ReturningString)



The following cell shows how your function is supposed to be used. The code prints out the extracted pairs for the first 1,000&nbsp;sentences in the data. It additionally numbers each pair with the identifier of the sentence (line number in the data file) which it was extracted from. Note that the sentence (line) numbering starts at index&nbsp;0.

In [7]:
#for i, doc in enumerate(nlp.pipe(data)):
 #   if i == 1000:
#        break
#    for person, org in extract(doc):
#        print("{}\t{}\t{}".format(i, person, org))

for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file, n=1000))):
    for person, org in extract(doc):
        print("{}\t{}\t{}".format(i, person, org))

207	Rugova	European Union
283	Michael Green	the U.S. National Security Council
351	Jendayi Frazer	Sudan Liberation Army
362	Dudley	TNK-BP
391	Mahmoud Abbas	Fatah
512	Kyi	the National League for Democracy
638	Hassan	CARE
802	Asif Ali Zardari	the Pakistan People's Party


Once you feel confident that your `extract` function does what it is supposed to do, execute the following cell to extract the entities from the full data set. Note that this will probably take a few minutes.

In [8]:
extracted = set()
for i, doc in enumerate(nlp.pipe(tm2.read_data(data_file))):
    for person, org in extract(doc):
        extracted.add((i, person, org))

After executing the above cell, all extracted id-string-string triples are in the set `extracted`. The code in the next cell will print the first 10&nbsp;triples in this set.

In [9]:
for i, person, org in sorted(extracted)[:10]:
    print("{}\t{}\t{}".format(i, person, org))

207	Rugova	European Union
283	Michael Green	the U.S. National Security Council
351	Jendayi Frazer	Sudan Liberation Army
362	Dudley	TNK-BP
391	Mahmoud Abbas	Fatah
512	Kyi	the National League for Democracy
638	Hassan	CARE
802	Asif Ali Zardari	the Pakistan People's Party
1349	Karen Hughes	State Department
1591	Fidel Castro	the Communist Party


## Problem 2: Evaluate your system

You now have an extractor, but how good is it? To help you answer this question, we provide you with a &lsquo;gold standard&rsquo; of entity pairs that your system should be able to extract. The following code loads them (again augmented with the relevant sentence id) from the file `gold.txt` and adds them to the set `gold`:

In [10]:
#gold_file = "C:/Users/Carles/Desktop/MasterStatistics-MachineLearning/Master_subjects/Text_Mining/Labs/Lab2/gold.txt"
#gold_file = "/home/carsa564/Desktop/TextMining/gold.txt"
gold_file = "/home/TDDE16/labs/l2/data/gold.txt"

gold = set()
with open(gold_file) as fp:
    for line in fp:
        columns = line.rstrip().split('\t')
        gold.add((int(columns[0]), columns[1], columns[2]))

The following code prints the 10&nbsp;first pairs from the gold standard:

In [11]:
for i, person, org in sorted(gold)[:10]:
    print("{}\t{}\t{}".format(i, person, org))


802	Ali Zardari	Pakistan People 's Party
2297	Abdul Aziz al-Hakim	Supreme Council
4823	Slavkov	Bulgarian National Olympic Committee
7902	Mr. Hakim	Supreme Council
8206	J. Patrick Boyle	American Meat Institute
8633	Ali Rodriguez	Petroleos de Venezuela
9004	Foreign Minister Joschka Fischer	Green Party
11021	Khalaf	al-Qaida
11259	Joseph Domenech	U.N. 's Food and Agricultural Organization
13043	David Petraeus	U.S. Central Command


Your task now is to write code that computes the precision, recall, and F1 measure of your extractor relative to the gold standard.

In [12]:
def evaluate(reference, predicted):
    """Print out the precision, recall, and F1 for the id-entity-entity
    triples in the set `predicted`, given the triples in the reference set.
    
    Args:
        reference: The reference set of triples.
        predicted: The set of predicted triples.
    Returns:
        Nothing, but prints out precision, recall, and F1.
        Precision: Nº words predicted as NN which are NN in gold standard /sum(total of NN of predicted)
        Recall:  Nº words predicted as NN which are NN in gold standard /sum(total of NN of Gold standard)
        F1 for the id-entity-entity: 2*Precision*Recall/(Precision + Recall)
    """
    #Calculating Length of both vectors
    PredictedLen = len(predicted)
    TrueLen = len(reference)
    
    #Calculating number of matches between both strings just by id
    
    TruePos = len(set.intersection(predicted , reference))
    
    Precision = TruePos/PredictedLen
    Recall = TruePos/TrueLen            
    F1 = 2*Precision*Recall/(Precision + Recall)
    print("P:{}\tR:{}\tF1:{}".format(Precision, Recall, F1))
    



In [13]:
idPredicted=[i for i, person, org in extracted]
idGold = [i for i, person, org in gold]
len(set.intersection(set(idPredicted), set(idGold)))
len(idGold)

46

The next cell shows how your function is intended to be used, as well as the suggested output format.

In [14]:
evaluate(gold, extracted)

P:0.014527845036319613	R:0.13043478260869565	F1:0.0261437908496732


## Problem 3: Entity resolution

Looking at the results of your quantitative evaluation, you will realise that your extractor (probably) does a rather poor job in matching the gold standard. One reason for this is that the NLP preprocessing is not perfect (spaCy was not trained on the annotations in the Groningen Meaning Bank), and that the approach of using regular expressions for relation extraction is rather naive.

Another reason however is that the current version of your system does not include a component for *entity resolution*. To give an example, your system does not realise that the strings `David Petraeus` and `General David Petraeus` refer to the same entity.

While writing an entity resolver is beyond the scope of this assignment, we ask you to *simulate* such a resolver. More specifically, you should implement a function `normalise` that takes an entity mention (a string) as its input and rewrites it to the form used in the gold standard. While in some sense this is &lsquo;cheating&rsquo;, it allows you to assess the performance of a more realistic system.

The following cell contains skeleton code for the `normalise` function.

In [15]:
def normalise(text):
    if text == "Asif Ali Zardari":
        return "Ali Zardari"
    if text == "the Supreme Council for the Islamic Revolution":
        return "Supreme Council"
    if text == "Joschka Fischer":
        return "Foreign Minister Joschka Fischer"
    if text == "U.N.":
        return "U.N. 's Food and Agricultural Organization"
    if text == "Lord":
        return "Lord 's Resistance Army"
    if text == "Chen Shui-bian":
        return "President Chen Shui-bian"
    if text == "Petraeus":
        return "General Petraeus"
    if text == "Fini":
        return "Mr. Fini"
    if text == "Mwanawasa":
        return "Mr. Mwanawasa"
    if text == "Abbas":
        return "Mr. Abbas"
    if text == "Coleman":
        return "Mr. Coleman"
    if text == "Ali":
        return "Prince Ali"
    if text == "Movement":
        return "Movement for Islamic Reform"
    if text == "David Petraeus":
        return "General David Petraeus"
    if text == "the Atomic Energy Organization":
        return "Atomic Energy Organization"   
    if text == "Lal Krishna":
        return "Lal Krishna Advani"
    if text == "Rafsanjani":
        return "Mr. Rafsanjani"
    return text.lstrip("the ") 
    

The next cell shows how `normalise` is intended to be used. Each triple in the set `extracted` is transformed by feeding the two entity mentions into the `normalise` function. The normalised triples are then added to a new set `extracted_normalised`.

In [16]:
extracted_normalised = set()
for triple in extracted:
    extracted_normalised.add((triple[0], normalise(triple[1]), normalise(triple[2])))

To pass the assignment, you should add enough normalisation rules to `normalise` to achieve a recall of at least 50%.

In [17]:
evaluate(gold, extracted_normalised)

P:0.06295399515738499	R:0.5652173913043478	F1:0.11328976034858387


## Problem 4: Limitations of the gold standard

Each entity pair in the gold standard has been manually checked for correctness. However, there is no guarantee that the gold standard contains all relevant pairs &ndash; there are in fact many pairs that are missing from the gold standard. Your last task in this assignment is to find at least 5&nbsp;entity pairs in the data that are valid instances of the &lsquo;is-leader-of&rsquo; relation but are not contained in the gold standard.

You can solve this task either by writing code or by manual work (inspecting the data file), or mix the two strategies. In any case, you should enter your pairs in the textbox below. Use the triple format shown above where for each pair you also specify the sentence id (line number in the data file) from which the instance was extracted.

In [18]:
## Finding all matches that are not gold standard 
idPredicted=[i for i, person, org in extracted]
idGold = [i for i, person, org in gold]
ids = set(idPredicted) - set(idGold)#Opposite to intersect

for i, person, org in sorted(extracted):
    for index in ids:
        if index == i:
            print(i, person, org)
            

207 Rugova European Union
283 Michael Green the U.S. National Security Council
351 Jendayi Frazer Sudan Liberation Army
362 Dudley TNK-BP
391 Mahmoud Abbas Fatah
512 Kyi the National League for Democracy
638 Hassan CARE
1349 Karen Hughes State Department
1591 Fidel Castro the Communist Party
1790 Koizumi the United Nations
1966 Lech Walesa Solidarity
2350 Basayev Nalchik
2477 Ismail Haniyeh Fatah
2982 Musharraf Islamist
3053 John Gai Yoh the Sudanese Liberation Movement
3291 Krasniqi the Kosovo Protection Corps
3543 Hassan Care International
4324 Ma Zhenchuan the Beijing Municipal Public Security Bureau
4567 Bush the U.S. Justice Department
4692 Heliodoro Diaz House of Representatives
4699 Agim Ceku the Kosovo Protection Corps
4753 Junichiro Koizumi APEC
5046 Daniel Pearl Al-Qaida
5082 Gul the AK Party
5255 Ayman Nour Noaman Gomaa
5450 Nicolas Sarkozy Group of Eight and Group
5507 Li Jianzhong the Chinese Academy of Carrier Rocket Technology
5522 Kofi Annan the Security Council
5762 Yu

Finally we ask you to reflect on the limitations of the evaluation that you carried out in this lab and discuss the question: *How should systems for information extraction really be evaluated?*. Here are some starting points for your discussion.

* How could one create a better gold standard for this task?
* What do precision, recall, and F1 actually measure in this context?
* What measures would be more suitable to evaluate this task?
* What other ways of evaluating systems for information extraction can you think of?

Submit your discussion as a short text (ca. 250&nbsp;words). When presenting your arguments, link back to your own results and experience from this lab, and to concepts you have learned in the lectures or in other parts of the course.

In order to improve the gold standard, one could have linguistic professionals who evaluate all the sentences manually. Moreover, synonyms of the expression "is leader of " could be used. Also, we could evaluate sentences not only in the "PERSON" "ORG" sequence but also viceversa in order to extract more relations that are inversed. 

In this context, precision is the fraction between the predicted pairs which are matched in the gold standard  divided by the total predicted pairs; recall is the fraction between the predicted pairs which are matched in the gold standard  divided by the total entity pairs in the gold standard; and F1 score is a combination of both such that 2*Precision*Recall/(Precision + Recall). 

If we want to match all pairs from the gold standard, the recall is the best measure; if we wanted to evaluate how optimal our function is, then the precision is the measure we are looking for; and if we wanted to optimize both measures then F1 is what we are looking for. Another measure that could be used are the Youden's J statistic as it also takes into account True negatives (whereas F1 does not), or rather just reporting the confusion matrix to see the overall error scheme. Also, accuracy could be a measure that could be complementary to our current measures.

Other ways of evaluating systems for information extraction could be a comparison of different machine learning methods for pair finding,  for the purpose of evaluating results (in this case, similar results).

This is the end of the assignment.